In [ ]:
#%pip install git+https://github.com/huggingface/peft
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl 
#%pip install https://pypi.org/simple/ bitsandbytes

In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer ,BitsAndBytesConfig
import torch

### 4bit Nested

In [ ]:
# %%time
# bnb_config = BitsAndBytesConfig(  
#     load_in_4bit=True,
#     bnb_4bit_quant_type= "nf4", # normal float : a new 4bit datatype adapted for weights that have been initialized using a normal distribution. 
#     bnb_4bit_compute_dtype= torch.bfloat16, # Set the computation data type to bfloat16 instead of default float32
#     bnb_4bit_use_double_quant= True, # Nested quantization : where the quantization constants from the first quantization are quantized again. - no performance lost
# )


# config = PeftConfig.from_pretrained("FO-UA/adapt-llm-Misc-Fr-40Xr256Xa512")
# model_new = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat", device_map="cuda", quantization_config=bnb_config)
# model_new = PeftModel.from_pretrained(model_new, "FO-UA/adapt-llm-Misc-Fr-40Xr256Xa512") #.to("cuda")

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained("FO-UA/adapt-llm-Misc-Fr-40Xr256Xa512", trust_remote_code=True)
# tokenizer.padding_side = 'right'
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_eos_token = True
# tokenizer.add_bos_token, tokenizer.add_eos_token

### 4bit + TorchCompile

In [ ]:
%%time
bnb_config = BitsAndBytesConfig(  
    load_in_4bit=True,
    bnb_4bit_quant_type= "nf4", # normal float : a new 4bit datatype adapted for weights that have been initialized using a normal distribution. 
    bnb_4bit_compute_dtype= torch.bfloat16, # Set the computation data type to bfloat16 instead of default float32
    bnb_4bit_use_double_quant= False, # Nested quantization : where the quantization constants from the first quantization are quantized again. - no performance lost
)

config = PeftConfig.from_pretrained("FO-UA/adapt-llm-Misc-Fr-40Xr256Xa512")
model_new = AutoModelForCausalLM.from_pretrained("AdaptLLM/finance-chat", device_map="cuda", quantization_config=bnb_config)
model_new = PeftModel.from_pretrained(model_new, "FO-UA/adapt-llm-Misc-Fr-40Xr256Xa512") #.to("cuda")
model_new_compile = torch.compile(model_new)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("FO-UA/adapt-llm-Misc-Fr-40Xr256Xa512", trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

# List input : Input Preparation 

In [ ]:
import pandas as pd
data=pd.read_csv("/kaggle/input/misclass/New_test_M_anonym.csv")

json_data = data.to_json(orient='records')

L=json_data.split('},{')

L[0]=L[0][2:]
L[-1]=L[-1][:-2]


for i in range(len(L)) : 
    L[i]=" Instruction : Generate a Misclassification Fraud Scenario : \n Completion : {" + L[i] + " }"
    

data_2=pd.read_csv("/kaggle/input/misclass/New_train_M_anonym.csv")

json_data_2 = data_2.to_json(orient='records')

LL=json_data_2.split('},{')

LL[0]=LL[0][2:]
LL[-1]=LL[-1][:-2]

for i in range(len(L)) : 
    LL[i]=" Instruction : Generate a Misclassification Fraud Scenario : \n Completion : {" + LL[i] + " }"
    
    
L_full = L + LL
len(L_full)

# Batch function Definition & Generation

## V1 : Simple

In [ ]:
# def batch_generation(L) :
    
#     L_result=[]
#     for i in range(len(L)) :
#         user_input = (f'''{L[i]} 
        
#             Instruction : Generate a Misclassification Fraud scenario :           
#             ''' +'''
            
#             {
#             "PersonID": 
#             "Seniority_previous_company(N-2) (in days)": 
#             "Contract_in_group_duration (in days)": 
#             "Seniority_previous_company(N-1) (in days)": 
#             "Seniority_in_Group (in days)": 
#             "Contract_Date": 
#             "Leave_Date": 
#             "Entry_Date_Company_1": 
#             "Sex": 
#             "Contract Type": 
#             "CODENT": 
#             "Position": 
#             "Classification": 
#             "Work_form": 
#             "CODSOR": 
#             "Original_company": 
#             "Leave_Category": 
#             "Leave_Reason": 
#             "Transfer_Reason": 
#             "Transfer_company": 
#             "Explanation": 
#             }

#             Completion :  
            
#             ''')
#         initial_prompt = ''' A Misclassification fraud is a payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
#         Note that the actual year we are currently at is : 2024. 
#         I want it to be in json format.
       
#         '''

#         prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


#         inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new.device) #to("cuda") #.to(model_new.device)
#         outputs =model_new.generate(input_ids=inputs, 
#                              max_new_tokens=2000,  #1200 for 3 examples 
#                              do_sample=True,
#                              temperature=1.2
#                             )[0]

#         answer_start = int(inputs.shape[-1])
#         pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
#         L_result.append(pred)
        
#         # save str in file format
#         print(user_input)
#         file_name = 'Assis_2_Temp12_Misclass' + str(i) + '.txt'
#         with open( file_name , 'w') as f:
#             f.write(pred)
#         print(f' \n file {i} saved')
        
#         print(pred)
#         print('\n ------------------')
    
#     return L_result


## V2

In [ ]:
def batch_generation(L,modell) :
    
    L_result=[]
    for i in range(len(L)) :
        user_input = (f'''{L[i]} 
        
            Instruction : Generate a Misclassification Fraud scenario :           
            ''' +'''
            
            {
            "PersonID": 
            "Seniority_previous_company(N-2) (in days)": 
            "Contract_in_group_duration (in days)": 
            "Seniority_previous_company(N-1) (in days)": 
            "Seniority_in_Group (in days)": 
            "Contract_Date": 
            "Leave_Date": 
            "Entry_Date_Company_1": 
            "Sex": 
            "Contract Type": 
            "CODENT": 
            "Position": 
            "Classification": 
            "Work_form": 
            "CODSOR": 
            "Original_company": 
            "Leave_Category": 
            "Leave_Reason": 
            "Transfer_Reason": 
            "Transfer_company": 
            "Explanation": 
            }

    Show the generated Fraud scenario, in json format.
    
    ''')
        initial_prompt = ''' A Misclassification fraud is a payroll fraud type where the perpetrator misclassifies an employee of an enterprise in order for the employee to get higher or lower pay than what his diploma or seniority should allow him.
            Note that the actual year we are currently in is : 2024. 
            I want it to be in json format.

            These are a few things you should note :
            These are correspondances between values of Position and Classification.
            Position	Classification
            Supervisor	Supervisor
            Manager/Director	Executive C3
            Manager/Director	Executive C1
            Assimilated Executive	Assimilated Executive
            Higher Executive	Executive
            Employee	Employee E1
            Employee	Employee E3
            Employee	Employee E4
            Manager/Director	Assimilated Executive
            Employee	Employee E2
            Executive	Executive C2

            The hierarchy of Classification in an ascending order is as follows : 
            Employee E1 - Employee E2 - Employee E3 - Employee E4 - Supervisor - Executive C1 - Executive C2 - Executive C3 - Assimilated Executive

            Transfer Company can take values only from this list : 
            ["TMG", "EUR", "ITI", "PRO", "TWW", "SMS", "SBS", "ITM", "IIS", "ATX", "AFA"]

            When Transfer_Reason is 'Not Transferred', and Transfer_company is 'UNSPECIFIED', it's because the employee has not been trnasferred, so Transfer_company takes a default value of UNSCPECIFIED.

            Show the generated Fraud scenario, in json format.
            '''

        prompt = f"<s>[INST] <<SYS>>{initial_prompt}<</SYS>>\n\n{user_input} [/INST]"


        inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids.to(model_new_compile.device) #to("cuda") #.to(model_new.device)
        outputs =model_new_compile.generate(input_ids=inputs, 
                             max_new_tokens=3000,  #1200 for 3 examples 
                     do_sample=True,
                     temperature=1.2
                    )[0]

        answer_start = int(inputs.shape[-1])
        pred = tokenizer.decode(outputs[answer_start:], skip_special_tokens=True)
        L_result.append(pred)
        
        # save str in file format
        print(user_input)
        file_name = 'Assis_2_Temp12_Misclass' + str(i) + '.txt'
        with open( file_name , 'w') as f:
            f.write(pred)
        print(f' \n file {i} saved')
        
        print(pred)
        print('\n ------------------')
    
    return L_result


In [ ]:
# batch_generation(L_full,model_new)

In [ ]:
batch_generation(L_full,model_new_compile)